# Code Repeatability Part 2: Data Cleaning and Merging of Datasets

### Summary:

This notebook imports two Twitter datasets, tweets scraped from specific users and tweets scraped from hashtag searches, and cleans them. It also imports a pair of datasets containing financial data covering the daily and hourly prices of Bitcoin. The Twitter data has many variables that had to be altered to become useful and many variables that had to be dropped. After these Twitter datasets were cleaned, they were combined with the financial data in order to add target the variable (Bitcoin close price) to the data.

**List of steps taken in cleaning:** <br />
**1.** Drop irrelevant or redundant columns (we already did most of this by only scraping relevant columns) <br />
**2.** Extract the useful information from user, inReplyToUser, and mentionedUsers columns <br />
**3.** Use date column to generate other useful datetime columns such as day_of_week and month <br />
**4.** Create dummy variables to replace hashtags and cashtags with variables such as contains_#DOGE and contains_$TSLA <br />
**5.** Deal with NaNs

**After this feature extraction was completed, the datasets were joined and the final product is four datasets:** <br />
**cleaned_tweets** The daily tweets from both the user and hashtag searches going back to the start of 2019 with completed data carpentry <br />
**tweets_and_close_prices** adds the close price of each stock/crypto we are interested in to cleaned_tweets, using the date as the variable the twitter and financial datasets join on. <br />
**cleaned_tweets_hourly** The hourly tweets from both the user and hashtag searches going back to November, 2020 with completed data carpentry <br />
**tweets_and_close_prices_hourly** adds the close price of each stock/crypto we are interested in to cleaned_tweets, using the date and hour as the variable the twitter and financial datasets join on. <br />

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
import os
import datetime

In [2]:
# getting original path of working notebook
original_path = os.getcwd()

In [3]:
# change directory to data file
# The file path would obviously be different for other people
os.chdir('../../casestudy_data/group_9')

In [4]:
# import datasets from saved csvs
daily_stock = pd.read_csv("daily_stock_last5yr.csv")
hourly_stock = pd.read_csv("hourly_stock_last8mo.csv")
hashtag_tweets = pd.read_csv("hashtag_tweets.csv", usecols=range(1,18))
user_tweets = pd.read_csv("user_tweets.csv", usecols=range(1,18)) 

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# change working directory back to original path after reading in data
os.chdir('../../')
os.chdir(original_path)

In [6]:
# Combining all tweets in such a way that there are no duplicates
cleaned_tweets = pd.concat([user_tweets,hashtag_tweets]).drop_duplicates(subset='id').reset_index(drop=True)

# No longer need id column
cleaned_tweets = cleaned_tweets.drop("id", 1)

# Check dimensions
cleaned_tweets.shape

(448662, 16)

In [7]:
# Get a look at the data
cleaned_tweets.head()

,date,content,user,replyCount,retweetCount,likeCount,quoteCount,lang,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,keyword_search
0,2021-06-15 00:20:35+00:00,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...,"{'_type': 'snscrape.modules.twitter.User', 'us...",475,643,10123,75,en,1.404554e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,elonmusk
1,2021-06-14 09:22:23+00:00,@tobyliiiiiiiiii @f_rmnn Cool,"{'_type': 'snscrape.modules.twitter.User', 'us...",526,330,8210,19,en,1.404168e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,elonmusk
2,2021-06-14 09:15:58+00:00,@FOXSports Epic dog,"{'_type': 'snscrape.modules.twitter.User', 'us...",607,504,5163,50,en,1.404273e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,elonmusk
3,2021-06-14 08:22:48+00:00,K is Kindred,"{'_type': 'snscrape.modules.twitter.User', 'us...",12130,1766,34151,244,en,1.404353e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",NaN,NaN,NaN,NaN,NaN,elonmusk
4,2021-06-14 08:21:46+00:00,Could turn out to be one big Philip K Dick jok...,"{'_type': 'snscrape.modules.twitter.User', 'us...",7433,3281,60077,322,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,elonmusk


In [8]:
cleaned_tweets.shape

(448662, 16)

**User Feature Extraction**

The user, inReplyToUser, and mentionedUsers columns contain a lot of potential features. They are not useful in their current form, so we had to use regex to extract features such as username, verified, and description into their own columns.

In [9]:
# An entry in the user column
# The user column contains many potential features
cleaned_tweets.iloc[0,2]

"{'_type': 'snscrape.modules.twitter.User', 'username': 'elonmusk', 'id': 44196397, 'displayname': 'Elon Musk', 'description': '', 'rawDescription': '', 'descriptionUrls': None, 'verified': True, 'created': '2009-06-02T20:12:29+00:00', 'followersCount': 57085054, 'friendsCount': 107, 'statusesCount': 14519, 'favouritesCount': 9042, 'listedCount': 74217, 'mediaCount': 887, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1404334078388670466/DgO3WL4S_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/44196397/1576183471', 'url': 'https://twitter.com/elonmusk'}"

In [10]:
# An entry in the inReplyToUser column
# There aren't as many features here (probably because snscrape isn't fully functional yet),
# but the username and displayname are available
cleaned_tweets.iloc[0,9] 

"{'_type': 'snscrape.modules.twitter.User', 'username': 'MileyCyrus', 'id': 268414482, 'displayname': 'Miley Ray Cyrus', 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/MileyCyrus'}"

**It appears that the user column contains a lot of information. Interesting bits that should be extracted into their own columns include:** <br />
username - This is the Twitter username (ex: @elonmusk) <br />
displayname - This is the bold name displayed just above or next to the username on Twitter (ex: Elon Musk) <br />
description - The description found on the user's profile <br />
verified - Whether or not the user is verified (has a blue checkmark indicating they are the public figure they say they are) <br />
followersCount - Number of users who follow the user <br />
friendsCount - Number of users the user follows <br />
statusesCount - Number of tweets the user has posted <br />
favouritesCount - Number of tweets the user has liked <br />

In [11]:
# This method extracts a string from one column in a dataframe and puts it into a new column
# The symbols are used to define what to extract and their meanings can be found by googling Python Regex
# ?<= is a command saying look for what is after this
# .*? is a command saying get everything found within the parameters
# ?= is a command saying look for what is before this
# Putting it all together, I'm essentially searching for anything between two specific strings in user
# and extracting that to a new column
cleaned_tweets['username'] = cleaned_tweets['user'].str.extract('(?<=\'username\': \')(.*?)(?=\', \')')
cleaned_tweets['displayname'] = cleaned_tweets['user'].str.extract('(?<=\'displayname\': \')(.*?)(?=\', \')')
cleaned_tweets['description'] = cleaned_tweets['user'].str.extract('(?<=\'description\': \')(.*?)(?=\', \')')

# These don't have quotes, so the syntax is slightly different
cleaned_tweets['verified'] = cleaned_tweets['user'].str.extract('(?<=\'verified\': )(.*?)(?=, \')')
cleaned_tweets['followersCount'] = cleaned_tweets['user'].str.extract('(?<=\'followersCount\': )(.*?)(?=, \')')
cleaned_tweets['friendsCount'] = cleaned_tweets['user'].str.extract('(?<=\'friendsCount\': )(.*?)(?=, \')')
cleaned_tweets['statusesCount'] = cleaned_tweets['user'].str.extract('(?<=\'statusesCount\': )(.*?)(?=, \')')
cleaned_tweets['favoritesCount'] = cleaned_tweets['user'].str.extract('(?<=\'favouritesCount\': )(.*?)(?=, \')')

# User column is no longer necessary
cleaned_tweets = cleaned_tweets.drop("user", 1)
cleaned_tweets.head()

,date,content,replyCount,retweetCount,likeCount,quoteCount,lang,inReplyToTweetId,inReplyToUser,mentionedUsers,...,cashtags,keyword_search,username,displayname,description,verified,followersCount,friendsCount,statusesCount,favoritesCount
0,2021-06-15 00:20:35+00:00,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...,475,643,10123,75,en,1.404554e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",...,NaN,elonmusk,elonmusk,Elon Musk,,True,57085054,107,14519,9042
1,2021-06-14 09:22:23+00:00,@tobyliiiiiiiiii @f_rmnn Cool,526,330,8210,19,en,1.404168e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",...,NaN,elonmusk,elonmusk,Elon Musk,,True,57085054,107,14519,9042
2,2021-06-14 09:15:58+00:00,@FOXSports Epic dog,607,504,5163,50,en,1.404273e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",...,NaN,elonmusk,elonmusk,Elon Musk,,True,57085054,107,14519,9042
3,2021-06-14 08:22:48+00:00,K is Kindred,12130,1766,34151,244,en,1.404353e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",NaN,...,NaN,elonmusk,elonmusk,Elon Musk,,True,57085054,107,14519,9042
4,2021-06-14 08:21:46+00:00,Could turn out to be one big Philip K Dick jok...,7433,3281,60077,322,en,NaN,NaN,NaN,...,NaN,elonmusk,elonmusk,Elon Musk,,True,57085054,107,14519,9042


In [12]:
# Add columns inReplyToUsername and inReplyToDisplayname, using inReplyToUser and the method above
cleaned_tweets['in_reply_to_username'] = cleaned_tweets['inReplyToUser'].str.extract('(?<=\'username\': \')(.*?)(?=\', \')')
cleaned_tweets['in_reply_to_displayname'] = cleaned_tweets['inReplyToUser'].str.extract('(?<=\'displayname\': \')(.*?)(?=\', \')')

# inReplyToUser column is no longer necessary
cleaned_tweets = cleaned_tweets.drop("inReplyToUser", 1)
cleaned_tweets.head()

,date,content,replyCount,retweetCount,likeCount,quoteCount,lang,inReplyToTweetId,mentionedUsers,coordinates,...,username,displayname,description,verified,followersCount,friendsCount,statusesCount,favoritesCount,in_reply_to_username,in_reply_to_displayname
0,2021-06-15 00:20:35+00:00,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...,475,643,10123,75,en,1.404554e+18,"[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,...,elonmusk,Elon Musk,,True,57085054,107,14519,9042,MileyCyrus,Miley Ray Cyrus
1,2021-06-14 09:22:23+00:00,@tobyliiiiiiiiii @f_rmnn Cool,526,330,8210,19,en,1.404168e+18,"[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,...,elonmusk,Elon Musk,,True,57085054,107,14519,9042,tobyliiiiiiiiii,Toby Li
2,2021-06-14 09:15:58+00:00,@FOXSports Epic dog,607,504,5163,50,en,1.404273e+18,"[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,...,elonmusk,Elon Musk,,True,57085054,107,14519,9042,FOXSports,FOX Sports
3,2021-06-14 08:22:48+00:00,K is Kindred,12130,1766,34151,244,en,1.404353e+18,NaN,NaN,...,elonmusk,Elon Musk,,True,57085054,107,14519,9042,elonmusk,Elon Musk
4,2021-06-14 08:21:46+00:00,Could turn out to be one big Philip K Dick jok...,7433,3281,60077,322,en,NaN,NaN,NaN,...,elonmusk,Elon Musk,,True,57085054,107,14519,9042,NaN,NaN


In [13]:
# Get the usernames and displaynames for mentionedUsers
# We could only figure out how to do this for the first mentioned user
cleaned_tweets['first_mentioned_username'] = cleaned_tweets['mentionedUsers'].str.extract('(?<=\'username\': \')(.*?)(?=\', \')')
cleaned_tweets['first_mentioned_displayname'] = cleaned_tweets['mentionedUsers'].str.extract('(?<=\'displayname\': \')(.*?)(?=\', \')')

# mentionedUsers column is no longer necessary
cleaned_tweets = cleaned_tweets.drop("mentionedUsers", 1)
cleaned_tweets.tail()

,date,content,replyCount,retweetCount,likeCount,quoteCount,lang,inReplyToTweetId,coordinates,place,...,description,verified,followersCount,friendsCount,statusesCount,favoritesCount,in_reply_to_username,in_reply_to_displayname,first_mentioned_username,first_mentioned_displayname
448657,2019-01-05 17:55:02+00:00,this song is AMAZING! Check out Saved from Ayo...,0,0,0,0,en,NaN,NaN,NaN,...,facebook.com/wiseguylingo,False,3731,3378,22557,2430,NaN,NaN,NaN,NaN
448658,2019-01-05 02:34:57+00:00,From my recent appearance on Good Morning Euro...,0,1,3,0,en,NaN,NaN,NaN,...,"Foreign Policy/Mil expert. Appears on Fox, CNN...",False,2062,378,3408,1618,NaN,NaN,defpriorities,Defense Priorities
448659,2019-01-04 22:17:16+00:00,HAPPY CDAY TOO A REAL NIGGA \nBLESSED TOO SEE ...,0,0,0,0,en,NaN,NaN,NaN,...,For Booking : 817-721-0475 AlbertLeeMusic@Gmai...,False,1884,906,66073,1589,NaN,NaN,NaN,NaN
448660,2019-01-04 19:24:54+00:00,New conceptual framework w/o dz specific compe...,0,4,12,1,en,1.080685e+18,{'_type': 'snscrape.modules.twitter.Coordinate...,"{'_type': 'snscrape.modules.twitter.Place', 'f...",...,#RadOnc @UChicagoMed w/ #MedEd interest\nAlum ...,False,2344,3005,1601,2685,d_golden,Dan Golden MD MHPE,UBC,University of British Columbia
448661,2019-01-01 04:14:22+00:00,"Here’s to a healthy, joyful &amp; prosperous n...",0,1,1,0,en,NaN,NaN,NaN,...,Mom | Wife | Advocate in honor of my Beloved B...,False,774,913,4264,12140,NaN,NaN,NaN,NaN


### Datetime Cleaning

In this section, we reformat the date column to match the date column in the finance data for joining the datasets in the future. We also add features based on date such as month, hour, and day of week.

This section involves repeating many of the same steps twice, once for the hourly dataset and once for the daily dataset. From here on out, variables whose names contain the word "hourly" correspond to the hourly data and those that do not contain "hourly" correspond to the daily data.

In [14]:
hourly_stock.head() # The datetime format needed to match the financial data is yyyy-mm-dd

,Unnamed: 0,Open,High,Low,Close,Adj Close,Volume,ticker
0,2020-11-22 21:00:00+00:00,18510.294922,18575.474609,18473.914062,18575.183594,18575.183594,0,BTC-USD
1,2020-11-22 22:00:00+00:00,18575.218750,18610.132812,18505.962891,18606.273438,18606.273438,0,BTC-USD
2,2020-11-22 23:00:00+00:00,18606.335938,18606.335938,18371.343750,18371.343750,18371.343750,205533184,BTC-USD
3,2020-11-23 00:00:00+00:00,18370.017578,18468.765625,18138.376953,18145.677734,18145.677734,0,BTC-USD
4,2020-11-23 01:00:00+00:00,18145.539062,18197.230469,18000.796875,18177.880859,18177.880859,0,BTC-USD


In [15]:
# For some reason, the timestamp column is unnamed
# Rename it and localize the timezones
hourly_stock["timestamp"] = pd.DatetimeIndex(pd.to_datetime(hourly_stock['Unnamed: 0'], utc=True)).tz_localize(None)
hourly_stock = hourly_stock.drop("Unnamed: 0", 1)
hourly_stock.head()

,Open,High,Low,Close,Adj Close,Volume,ticker,timestamp
0,18510.294922,18575.474609,18473.914062,18575.183594,18575.183594,0,BTC-USD,2020-11-22 21:00:00
1,18575.218750,18610.132812,18505.962891,18606.273438,18606.273438,0,BTC-USD,2020-11-22 22:00:00
2,18606.335938,18606.335938,18371.343750,18371.343750,18371.343750,205533184,BTC-USD,2020-11-22 23:00:00
3,18370.017578,18468.765625,18138.376953,18145.677734,18145.677734,0,BTC-USD,2020-11-23 00:00:00
4,18145.539062,18197.230469,18000.796875,18177.880859,18177.880859,0,BTC-USD,2020-11-23 01:00:00


In [16]:
# The date range that the hourly tweets and hourly stock datasets have in common is 11/3/20-6/14/21
# Drop dates not in this range

hourly_stock = hourly_stock[pd.to_datetime(hourly_stock["timestamp"]) >= datetime.datetime(2020, 11, 3)]
hourly_stock = hourly_stock[pd.to_datetime(hourly_stock["timestamp"]) < datetime.datetime(2021, 6, 15)]

In [17]:
# Here, we create two columns representing the date
# One in the same format as the financial data for immediate use
# The other preserves the timestamp in its current form, but as a datetime object
cleaned_tweets_hourly = cleaned_tweets
cleaned_tweets_hourly['timestamp'] = pd.DatetimeIndex(pd.to_datetime(cleaned_tweets_hourly['date'], utc=True)).tz_localize(None)
cleaned_tweets_hourly['Date'] = cleaned_tweets_hourly['timestamp'].dt.strftime('%Y-%m-%d') # Add a column whose format matches finance data
cleaned_tweets_hourly['Date'] = pd.to_datetime(cleaned_tweets_hourly['Date'])
cleaned_tweets_hourly = cleaned_tweets_hourly.drop('date', 1) # Remove original column because it's redundant
cleaned_tweets_hourly.head()

,content,replyCount,retweetCount,likeCount,quoteCount,lang,inReplyToTweetId,coordinates,place,hashtags,...,followersCount,friendsCount,statusesCount,favoritesCount,in_reply_to_username,in_reply_to_displayname,first_mentioned_username,first_mentioned_displayname,timestamp,Date
0,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...,475,643,10123,75,en,1.404554e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,MileyCyrus,Miley Ray Cyrus,MileyCyrus,Miley Ray Cyrus,2021-06-15 00:20:35,2021-06-15
1,@tobyliiiiiiiiii @f_rmnn Cool,526,330,8210,19,en,1.404168e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,tobyliiiiiiiiii,Toby Li,tobyliiiiiiiiii,Toby Li,2021-06-14 09:22:23,2021-06-14
2,@FOXSports Epic dog,607,504,5163,50,en,1.404273e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,FOXSports,FOX Sports,FOXSports,FOX Sports,2021-06-14 09:15:58,2021-06-14
3,K is Kindred,12130,1766,34151,244,en,1.404353e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,elonmusk,Elon Musk,NaN,NaN,2021-06-14 08:22:48,2021-06-14
4,Could turn out to be one big Philip K Dick jok...,7433,3281,60077,322,en,NaN,NaN,NaN,NaN,...,57085054,107,14519,9042,NaN,NaN,NaN,NaN,2021-06-14 08:21:46,2021-06-14


In [18]:
# Add a bunch of useful datetime variables
cleaned_tweets_hourly['month'] = cleaned_tweets_hourly['timestamp'].dt.month
cleaned_tweets_hourly['year'] = cleaned_tweets_hourly['timestamp'].dt.year
cleaned_tweets_hourly['day'] = cleaned_tweets_hourly['timestamp'].dt.day
cleaned_tweets_hourly['day_of_week'] = cleaned_tweets_hourly['timestamp'].dt.dayofweek
cleaned_tweets_hourly['hour'] = cleaned_tweets_hourly['timestamp'].dt.hour

In [19]:
# Round timestamp to the hour to match with finance data
cleaned_tweets["timestamp"] = cleaned_tweets["timestamp"].dt.round('H')

In [20]:
# Remove tweets from outside of our date range
cleaned_tweets_hourly = cleaned_tweets_hourly[pd.to_datetime(cleaned_tweets_hourly["timestamp"]) >= datetime.datetime(2020, 11, 3)]
cleaned_tweets_hourly = cleaned_tweets_hourly[pd.to_datetime(cleaned_tweets_hourly["timestamp"]) < datetime.datetime(2021, 6, 15)]

**Repeat for daily data**

In [21]:
daily_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2016-07-22,664.921997,666.583008,646.721985,650.619019,650.619019,134169000,BTC-USD
1,2016-07-23,650.726013,656.366028,648.523987,655.556030,655.556030,69532200,BTC-USD
2,2016-07-24,655.409973,663.109985,652.793030,661.284973,661.284973,118184000,BTC-USD
3,2016-07-25,661.263000,661.828003,653.395020,654.096985,654.096985,78176496,BTC-USD
4,2016-07-26,654.226013,656.224976,645.879028,651.783997,651.783997,225135008,BTC-USD


In [22]:
cleaned_tweets['timestamp'] = pd.to_datetime(cleaned_tweets['date']) # Give more appropriate name and change to datetime
cleaned_tweets['Date'] = cleaned_tweets['timestamp'].dt.strftime('%Y-%m-%d') # Add a column whose format matches finance data
cleaned_tweets['Date'] = pd.to_datetime(cleaned_tweets['Date'])
cleaned_tweets = cleaned_tweets.drop('date', 1) # Remove original column because it's redundant
cleaned_tweets.head()

,content,replyCount,retweetCount,likeCount,quoteCount,lang,inReplyToTweetId,coordinates,place,hashtags,...,followersCount,friendsCount,statusesCount,favoritesCount,in_reply_to_username,in_reply_to_displayname,first_mentioned_username,first_mentioned_displayname,timestamp,Date
0,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...,475,643,10123,75,en,1.404554e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,MileyCyrus,Miley Ray Cyrus,MileyCyrus,Miley Ray Cyrus,2021-06-15 00:20:35+00:00,2021-06-15
1,@tobyliiiiiiiiii @f_rmnn Cool,526,330,8210,19,en,1.404168e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,tobyliiiiiiiiii,Toby Li,tobyliiiiiiiiii,Toby Li,2021-06-14 09:22:23+00:00,2021-06-14
2,@FOXSports Epic dog,607,504,5163,50,en,1.404273e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,FOXSports,FOX Sports,FOXSports,FOX Sports,2021-06-14 09:15:58+00:00,2021-06-14
3,K is Kindred,12130,1766,34151,244,en,1.404353e+18,NaN,NaN,NaN,...,57085054,107,14519,9042,elonmusk,Elon Musk,NaN,NaN,2021-06-14 08:22:48+00:00,2021-06-14
4,Could turn out to be one big Philip K Dick jok...,7433,3281,60077,322,en,NaN,NaN,NaN,NaN,...,57085054,107,14519,9042,NaN,NaN,NaN,NaN,2021-06-14 08:21:46+00:00,2021-06-14


In [23]:
# Add a bunch of useful datetime variables
# All the same as above, except without the hour column this time
cleaned_tweets['month'] = cleaned_tweets['timestamp'].dt.month
cleaned_tweets['year'] = cleaned_tweets['timestamp'].dt.year
cleaned_tweets['day'] = cleaned_tweets['timestamp'].dt.day
cleaned_tweets['day_of_week'] = cleaned_tweets['timestamp'].dt.dayofweek

In [24]:
cleaned_tweets.head()

,content,replyCount,retweetCount,likeCount,quoteCount,lang,inReplyToTweetId,coordinates,place,hashtags,...,in_reply_to_username,in_reply_to_displayname,first_mentioned_username,first_mentioned_displayname,timestamp,Date,month,year,day,day_of_week
0,@MileyCyrus 133T H4X0R got my phone. Nuthin I ...,475,643,10123,75,en,1.404554e+18,NaN,NaN,NaN,...,MileyCyrus,Miley Ray Cyrus,MileyCyrus,Miley Ray Cyrus,2021-06-15 00:20:35+00:00,2021-06-15,6,2021,15,1
1,@tobyliiiiiiiiii @f_rmnn Cool,526,330,8210,19,en,1.404168e+18,NaN,NaN,NaN,...,tobyliiiiiiiiii,Toby Li,tobyliiiiiiiiii,Toby Li,2021-06-14 09:22:23+00:00,2021-06-14,6,2021,14,0
2,@FOXSports Epic dog,607,504,5163,50,en,1.404273e+18,NaN,NaN,NaN,...,FOXSports,FOX Sports,FOXSports,FOX Sports,2021-06-14 09:15:58+00:00,2021-06-14,6,2021,14,0
3,K is Kindred,12130,1766,34151,244,en,1.404353e+18,NaN,NaN,NaN,...,elonmusk,Elon Musk,NaN,NaN,2021-06-14 08:22:48+00:00,2021-06-14,6,2021,14,0
4,Could turn out to be one big Philip K Dick jok...,7433,3281,60077,322,en,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2021-06-14 08:21:46+00:00,2021-06-14,6,2021,14,0


In [25]:
cleaned_tweets.shape

(448662, 30)

In [26]:
# Hourly dataset contains one more additional feature, hour
cleaned_tweets_hourly.shape

(241140, 31)

**Location cleaning**

In [27]:
# Remove location features because they aren't useful
# Only a very small percentage of the tweets contain location data
cleaned_tweets = cleaned_tweets.drop(['coordinates', 'place'], 1)
cleaned_tweets_hourly = cleaned_tweets_hourly.drop(['coordinates', 'place'], 1)

**Hashtag/Cashtag cleaning**

In [28]:
# Hashtags appear to be stored in arrays
user_tweets['hashtags'].unique() 

array([nan, "['Bitcoin']", "['Mars']", ..., "['DoYourResearch']",
       "['DoOnlyGoodEveryday', 'Rakuen']", "['MayTheFourthBeWithYou']"],
      dtype=object)

In [29]:
# Add True/False variables indicating the appearance of relevant hashtags and cashtags
# Cashtags function the same way as hashtags, but are used exclusively for financial ticker symbols
cleaned_tweets['contains_#TSLA'] = cleaned_tweets['hashtags'].str.contains('TSLA', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_#BTC'] = cleaned_tweets['hashtags'].str.contains('BTC', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_#DOGE'] = cleaned_tweets['hashtags'].str.contains('DOGE', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_#GME'] = cleaned_tweets['hashtags'].str.contains('GME', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_#AMC'] = cleaned_tweets['hashtags'].str.contains('AMC', na=False, flags=re.IGNORECASE)

cleaned_tweets['contains_#tesla'] = cleaned_tweets['hashtags'].str.contains('tesla', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_#bitcoin'] = cleaned_tweets['hashtags'].str.contains('bitcoin', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_#dogecoin'] = cleaned_tweets['hashtags'].str.contains('dogecoin', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_#gamestop'] = cleaned_tweets['hashtags'].str.contains('gamestop', na=False, flags=re.IGNORECASE)
cleaned_tweets['no_hashtag'] = cleaned_tweets['hashtags'].isna()

cleaned_tweets['contains_$TSLA'] = cleaned_tweets['cashtags'].str.contains('TSLA', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_$BTC'] = cleaned_tweets['cashtags'].str.contains('BTC', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_$DOGE'] = cleaned_tweets['cashtags'].str.contains('DOGE', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_$GME'] = cleaned_tweets['cashtags'].str.contains('GME', na=False, flags=re.IGNORECASE)
cleaned_tweets['contains_$AMC'] = cleaned_tweets['cashtags'].str.contains('AMC', na=False, flags=re.IGNORECASE)
cleaned_tweets['no_cashtag'] = cleaned_tweets['cashtags'].isna()

In [30]:
# Do the same for the hourly data
cleaned_tweets_hourly['contains_#TSLA'] = cleaned_tweets_hourly['hashtags'].str.contains('TSLA', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_#BTC'] = cleaned_tweets_hourly['hashtags'].str.contains('BTC', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_#DOGE'] = cleaned_tweets_hourly['hashtags'].str.contains('DOGE', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_#GME'] = cleaned_tweets_hourly['hashtags'].str.contains('GME', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_#AMC'] = cleaned_tweets_hourly['hashtags'].str.contains('AMC', na=False, flags=re.IGNORECASE)

cleaned_tweets_hourly['contains_#tesla'] = cleaned_tweets_hourly['hashtags'].str.contains('tesla', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_#bitcoin'] = cleaned_tweets_hourly['hashtags'].str.contains('bitcoin', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_#dogecoin'] = cleaned_tweets_hourly['hashtags'].str.contains('dogecoin', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_#gamestop'] = cleaned_tweets_hourly['hashtags'].str.contains('gamestop', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['no_hashtag'] = cleaned_tweets_hourly['hashtags'].isna()

cleaned_tweets_hourly['contains_$TSLA'] = cleaned_tweets_hourly['cashtags'].str.contains('TSLA', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_$BTC'] = cleaned_tweets_hourly['cashtags'].str.contains('BTC', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_$DOGE'] = cleaned_tweets_hourly['cashtags'].str.contains('DOGE', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_$GME'] = cleaned_tweets_hourly['cashtags'].str.contains('GME', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['contains_$AMC'] = cleaned_tweets_hourly['cashtags'].str.contains('AMC', na=False, flags=re.IGNORECASE)
cleaned_tweets_hourly['no_cashtag'] = cleaned_tweets_hourly['cashtags'].isna()

In [31]:
# The information we need has been extracted, so these columns are no longer necessary
cleaned_tweets = cleaned_tweets.drop(['hashtags', 'cashtags'], 1)
cleaned_tweets_hourly = cleaned_tweets_hourly.drop(['hashtags', 'cashtags'], 1)

**Simplify the dataset and fill NaNs**

In [32]:
# Remove unnecessary column
cleaned_tweets = cleaned_tweets.drop('inReplyToTweetId', 1)
cleaned_tweets_hourly = cleaned_tweets_hourly.drop('inReplyToTweetId', 1)

In [33]:
cleaned_tweets.columns[cleaned_tweets.isna().any()].tolist()

['displayname',
 'description',
 'in_reply_to_username',
 'in_reply_to_displayname',
 'first_mentioned_username',
 'first_mentioned_displayname']

In [34]:
cleaned_tweets_hourly.columns[cleaned_tweets_hourly.isna().any()].tolist()

['displayname',
 'description',
 'in_reply_to_username',
 'in_reply_to_displayname',
 'first_mentioned_username',
 'first_mentioned_displayname']

In [35]:
# After searching some of these users, there is no apparent reason why the displayname didn't come up
# Since it's only 2,000 of  400,000 tweets, we'll just remove these rows

#cleaned_tweets[cleaned_tweets['displayname'].isna()]
cleaned_tweets.shape

(448662, 41)

In [36]:
cleaned_tweets = cleaned_tweets[cleaned_tweets['displayname'].notna()]
cleaned_tweets.shape

(446686, 41)

In [37]:
# Do the same for the hourly dataset
cleaned_tweets_hourly = cleaned_tweets_hourly[cleaned_tweets_hourly['displayname'].notna()]

In [38]:
# Fill remaining NaNs with empty space or "None"
# These NaNs mean the user didn't reply or mention anyone and they don't have a description on their account
values = {'description': ' ', 'in_reply_to_username': 'None', 'in_reply_to_displayname': 'None',
          'first_mentioned_username': 'None', 'first_mentioned_displayname': 'None'}
cleaned_tweets = cleaned_tweets.fillna(value=values)
cleaned_tweets_hourly = cleaned_tweets_hourly.fillna(value=values)

In [39]:
# No more NaNs
cleaned_tweets.columns[cleaned_tweets.isna().any()].tolist()

[]

In [40]:
# No more NaNs
cleaned_tweets_hourly.columns[cleaned_tweets_hourly.isna().any()].tolist()

[]

**Converting financial data to a form that can be combined with Twitter data**

In [41]:
daily_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2016-07-22,664.921997,666.583008,646.721985,650.619019,650.619019,134169000,BTC-USD
1,2016-07-23,650.726013,656.366028,648.523987,655.556030,655.556030,69532200,BTC-USD
2,2016-07-24,655.409973,663.109985,652.793030,661.284973,661.284973,118184000,BTC-USD
3,2016-07-25,661.263000,661.828003,653.395020,654.096985,654.096985,78176496,BTC-USD
4,2016-07-26,654.226013,656.224976,645.879028,651.783997,651.783997,225135008,BTC-USD


In [42]:
# Reformat the dataset so that each row contains the date and the close price of every stock/crypto
daily_close_prices = daily_stock[['Date', 'Close', 'ticker']] # Keep only target variable and date
daily_close_prices = daily_close_prices.pivot(index="Date", columns="ticker", values="Close")

# The stocks have NaNs for their close prices because stocks are only traded on weekdays
# To solve this, just fill missing values with the previous day's close price
daily_close_prices = daily_close_prices.ffill()

# Convert back to a dataframe so it can be combined with the Twitter data
daily_close_prices = pd.DataFrame(daily_close_prices.to_records())
daily_close_prices.head()

,Date,BTC-USD
0,2016-07-22,650.619019
1,2016-07-23,655.556030
2,2016-07-24,661.284973
3,2016-07-25,654.096985
4,2016-07-26,651.783997


In [43]:
# Reformat the dataset so that each row contains the date and the close price of every stock/crypto
hourly_close_prices = hourly_stock[['timestamp', 'Close', 'ticker']] # Keep only target variable and date
hourly_close_prices = hourly_close_prices.pivot(index="timestamp", columns="ticker", values= "Close")

# The stocks have NaNs for their close prices because stocks are only traded on weekdays
# To solve this, just fill missing values with the previous day's close price
hourly_close_prices = hourly_close_prices.ffill()

# Convert back to a dataframe so it can be combined with the Twitter data
hourly_close_prices = pd.DataFrame(hourly_close_prices.to_records())
hourly_close_prices.head()

,timestamp,BTC-USD
0,2020-11-22 21:00:00,18575.183594
1,2020-11-22 22:00:00,18606.273438
2,2020-11-22 23:00:00,18371.343750
3,2020-11-23 00:00:00,18145.677734
4,2020-11-23 01:00:00,18177.880859


### Adding the target variable to the datasets

In [44]:
# Adding the close prices for each stock/crypto to the tweet data
# Using a left join on Date
daily_close_prices['Date'] = pd.to_datetime(daily_close_prices['Date'])
cleaned_tweets['Date'] = pd.to_datetime(cleaned_tweets['Date'])
tweets_and_close_prices = pd.merge(cleaned_tweets, daily_close_prices, on='Date', how='left')
tweets_and_close_prices.shape

(446686, 42)

In [45]:
# Do the same for hourly data
hourly_close_prices['timestamp'] = pd.to_datetime(hourly_close_prices['timestamp'])
cleaned_tweets_hourly['timestamp'] = pd.to_datetime(cleaned_tweets_hourly['timestamp'])
tweets_and_close_prices_hourly = pd.merge(cleaned_tweets_hourly, hourly_close_prices, on='timestamp', how='left')
tweets_and_close_prices_hourly.shape

(239709, 43)

### Saving combined datasets as csvs

In [46]:
os.chdir('../../casestudy_data/group_9')

In [47]:
# Cleaned daily and hourly datasets both with and without the target variables added
cleaned_tweets.to_csv("all_cleaned_tweets.csv")
cleaned_tweets_hourly.to_csv("all_cleaned_tweets_hourly_8months.csv")
tweets_and_close_prices.to_csv("cleaned_tweets_and_close_prices.csv")
tweets_and_close_prices_hourly.to_csv("cleaned_tweets_and_close_prices_hourly_8months.csv")